In [0]:
columns=["Employee_id","Employee Name", "Gender","Salary"]
emp_data=(
    (1,"Irfan", "M", 120000),
    (2,"Irfana", "F", 110000),
    (3,"Nikgil", "", 120000),
    (4,"XYZ", None , 80000)
)

In [0]:
emp_df=spark.createDataFrame(emp_data, columns)
display(emp_df)

Employee_id,Employee Name,Gender,Salary
1,Irfan,M,120000
2,Irfana,F,110000
3,Nikgil,,120000
4,XYZ,null,80000


In [0]:
from pyspark.sql.functions import when
emp_df=emp_df.withColumn("Full_Gender", when(emp_df.Gender == "M", "Male")
                                        .when(emp_df.Gender == "F", "Female")
                                        .when(emp_df.Gender.isNull(), "")
                                        .otherwise(emp_df.Gender))
display(emp_df)

Employee_id,Employee Name,Gender,Salary,Full_Gender
1,Irfan,M,120000,Male
2,Irfana,F,110000,Female
3,Nikgil,,120000,
4,XYZ,null,80000,


In [0]:
from pyspark.sql.functions import expr
emp_df1=emp_df.withColumn("Sal_Greater_100000",expr("""case when Salary > 100000 Then "Yes"
                                                         Else "NO"
                                                         end"""))

In [0]:
display(emp_df1)

Employee_id,Employee Name,Gender,Salary,Full_Gender,Sal_Greater_100000
1,Irfan,M,120000,Male,Yes
2,Irfana,F,110000,Female,Yes
3,Nikgil,,120000,,Yes
4,XYZ,null,80000,,NO


In [0]:
from pyspark.sql.functions import lit
emp_df1=emp_df1.withColumn("manager_id", lit(234))
display(emp_df1)

Employee_id,Employee Name,Gender,Salary,Full_Gender,Sal_Greater_100000,manager_id
1,Irfan,M,120000,Male,Yes,234
2,Irfana,F,110000,Female,Yes,234
3,Nikgil,,120000,,Yes,234
4,XYZ,null,80000,,NO,234


In [0]:
display(emp_df[emp_df.Employee_id == 3])

Employee_id,Employee Name,Gender,Salary,Full_Gender
3,Nikgil,,120000,


In [0]:
from pyspark.sql.functions import desc
emp_df=emp_df.orderBy(desc("Salary"))

In [0]:
display(emp_df.select(max("Salary").alias("max_sal")))

Column<'max_sal'>

In [0]:
from pyspark.sql.functions import max, when
display(emp_df[emp_df.Salary!= emp_df.select(max("Salary").alias("max_sal")).max_sal])

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2841663187176834>:2
      1 from pyspark.sql.functions import max, when
----> 2 display(emp_df[emp_df.Salary!= emp_df.select(max("Salary").alias("max_sal")).max_sal])

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:2921, in DataFrame.__getitem__(self, item)
   2919     return Column(jc)
   2920 elif isinstance(item, Column):
-> 2921     return self.filter(item)
   2922 elif isinstance(item, (list, tuple)):
   2923     return self.select(*item)

File

In [0]:
columns=["Employee_id","Employee Name", "Gender","Salary", "department"]
emp_data=(
    (1,"Irfan", "M", 120000, "CSE"),
    (2,"Irfana", "F", 110000,"CSE"),
    (3,"Nikgil", "", 120000,"ECE"),
    (4,"XYZ", None , 180000,"ME"),
    (5,"ABC", None , 280000,"ME"),
    (9,"ABC2", None , 280000,"ME"),
    (6,"asd", None , 120000,"MS"),
    (7,"XYZasd", None , 380000,"MS"),
)
emp_df=spark.createDataFrame(emp_data,columns)

In [0]:
display(emp_df)

Employee_id,Employee Name,Gender,Salary,department
1,Irfan,M,120000,CSE
2,Irfana,F,110000,CSE
3,Nikgil,,120000,ECE
4,XYZ,null,180000,ME
5,ABC,null,280000,ME
9,ABC2,null,280000,ME
6,asd,null,120000,MS
7,XYZasd,null,380000,MS


In [0]:
from pyspark.sql.functions import desc, rank, dense_rank
from pyspark.sql.window import Window
emp_df=emp_df.withColumn("Rank",dense_rank().over(Window.partitionBy("department").orderBy(desc("Salary"))))
display(emp_df)

Employee_id,Employee Name,Gender,Salary,department,Rank
1,Irfan,M,120000,CSE,1
2,Irfana,F,110000,CSE,2
3,Nikgil,,120000,ECE,1
5,ABC,null,280000,ME,1
9,ABC2,null,280000,ME,1
4,XYZ,null,180000,ME,2
7,XYZasd,null,380000,MS,1
6,asd,null,120000,MS,2


In [0]:
display(emp_df.filter("Rank=2"))

Employee_id,Employee Name,Gender,Salary,department,Rank
2,Irfana,F,110000,CSE,2
4,XYZ,null,180000,ME,2
6,asd,null,120000,MS,2


In [0]:
from pyspark.sql.functions import desc, rank, dense_rank, row_number
from pyspark.sql.window import Window
emp_df=emp_df.withColumn("row_number",row_number().over(Window.partitionBy("department").orderBy(desc("Salary"))))
display(emp_df)

Employee_id,Employee Name,Gender,Salary,department,Rank,row_number
1,Irfan,M,120000,CSE,1,1
2,Irfana,F,110000,CSE,2,2
3,Nikgil,,120000,ECE,1,1
5,ABC,null,280000,ME,1,1
9,ABC2,null,280000,ME,1,2
4,XYZ,null,180000,ME,2,3
7,XYZasd,null,380000,MS,1,1
6,asd,null,120000,MS,2,2
